In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os, os.path, shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pathlib
import os
import datetime
from PIL import Image
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

## Data acquisition and preprocessing

#### Download and load food-101 dataset, excluding worst food classes which were previously determined

In [ ]:
# Splits dataset into training folder and test folder

!wget https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar -xzf food-101.tar.gz
data_dir = pathlib.Path("/content/food-101/images")
val_dir = pathlib.Path("/content/validation")
test_dir = pathlib.Path("/content/test")

exception = [42, 18, 84, 3, 17, 47, 80, 36, 26, 89, 96, 39, 5, 49, 57, 59, 99, 8, 10, 0, 15, 82, 56, 67, 37, 93, 22, 50, 4, 87, 77]

folder_path = data_dir
train_path = test_dir

folders = sorted([f for f in os.listdir(folder_path)])

for index, folder in enumerate(folders):
  path = os.path.join(folder_path, folder)
  tpath = os.path.join(train_path,folder)

  if index in exception:
    shutil.rmtree(path)
    continue

  if not os.path.exists(tpath):
      os.makedirs(tpath)

  images = os.listdir(path)
  count = 0
  for image in images:
    if count <= 100:
      old_image_path = os.path.join(path,image)
      new_image_path = os.path.join(tpath,image)
      shutil.move(old_image_path, new_image_path)
    count += 1

In [ ]:
# Split again to create validation set

folder_path = data_dir
val_path = val_dir

folders = [f for f in os.listdir(folder_path)]

for folder in folders:
    path = os.path.join(folder_path, folder)
    tpath = os.path.join(val_path,folder)

    if not os.path.exists(tpath):
        os.makedirs(tpath)

    images = os.listdir(path)
    count = 0
    for image in images:
      if count <= 100:
        old_image_path = os.path.join(path,image)
        new_image_path = os.path.join(tpath,image)
        shutil.move(old_image_path, new_image_path)
      count += 1

#### Define data generators to apply transformations to data, and create train and validation datasets

In [ ]:
batch_size = 64
img_height = 256
img_width = 256

train_img_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=5,
                                   horizontal_flip=True,
                                   vertical_flip=False,
                                   fill_mode='nearest',
                                   zoom_range=0.2
                                  )

val_img_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_ds = train_img_data_gen.flow_from_directory(directory=data_dir,
                                                    class_mode='sparse',
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    shuffle=True
                                                   )

val_ds = val_img_data_gen.flow_from_directory(directory=val_dir,
                                                    class_mode='sparse',
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    shuffle=True
                                                   )

Found 62930 images belonging to 70 classes.
Found 7070 images belonging to 70 classes.


#### Visualize first 5 preprocessed images

In [ ]:
x_batch, y_batch = next(train_ds)

for i in range(5):
    image = x_batch[i]
    plt.imshow(image)
    plt.show()

## Model creation and training

#### Create model using pretrained portion of InceptionResNetV2, and freeze pretrained layers

In [ ]:
num_classes = 70

modelBase = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3), pooling='avg')

modelp = modelBase.output
modelp = Dense(2048, kernel_initializer='he_uniform', activation='relu')(modelp)
output = Dense(num_classes, activation='softmax')(modelp)

model = Model(inputs=modelBase.input, outputs=output)

for layer in modelBase.layers:
    layer.trainable = False

87916544/87910968 [==============================] - 2s 0us/step


In [ ]:
# Unfreeze top few layers

for layer in model.layers[:777]:
   layer.trainable = False
for layer in model.layers[777:]:
   layer.trainable = True

#### Define learning rate and begin training

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(
  optimizer=opt,
  loss=tf.losses.SparseCategoricalCrossentropy(),
  metrics=['sparse_categorical_accuracy'])

In [ ]:
# Saves model weights when new validation low observed.
# Learning rate schedule will reduce learnign rate by 0.1
# after 5 epochs of no val_loss improvement.

print("Start training:",datetime.datetime.now())

modelPath = "/content/drive/MyDrive/MAIS/MAIS202Data/inception-new3"

earlyStopping = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='min')
mcp_save = ModelCheckpoint(modelPath, save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min')

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    verbose=1,
    initial_epoch=0,
    max_queue_size=100,
    workers=300,
    use_multiprocessing=False,
    callbacks=[earlyStopping, mcp_save, reduce_lr_loss]
)

## Model Testing

In [ ]:
# Preprocess test dataset

test_ds = val_img_data_gen.flow_from_directory(directory=test_dir,
                                                    class_mode='sparse',
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    shuffle=True
                                                   )

In [ ]:
print("Evaluate on test data")
results = model.evaluate(test_ds, verbose=1)
print("test loss, test acc:", results)

Evaluate on test data
111/111 [==============================] - 33s 297ms/step - loss: 0.2155 - sparse_categorical_accuracy: 0.9375
test loss, test acc: [0.21546627581119537, 0.9374822974205017]


## Convert model to .mlmodel and download to drive

In [ ]:
!pip install coremltools==4.0
import coremltools as ct

In [ ]:
%cd /content/drive/MyDrive/
model.save('ImageClassifier.h5')

output_labels = list(test_ds.class_indices.keys())

classifier_config = ct.ClassifierConfig(output_labels)

image_input = ct.ImageType(shape=(1, 256, 256, 3,),
                           bias=[-1,-1,-1], scale=2.0/225.0)

image_classifier = ct.convert('ImageClassifier.h5', inputs=[image_input], classifier_config=classifier_config)

image_classifier.short_description = 'Classification of 70 different foods'
image_classifier.input_description["input_1"] = "Input image to be classified"
image_classifier.output_description["classLabel"] = "Most likely image category"

image_classifier.save('ImageClassifier.mlmodel')